In [ ]:
## 데이터 없는 테이블 가져오기

In [1]:
import pymysql
from datetime import datetime
import json
import os

def get_connection():
    host = input("DB 호스트: ")
    user = input("DB 사용자명: ")
    password = input("DB 비밀번호: ")
    db = input("DB 이름: ")
    port = int(input("포트 (기본 3306): ") or "3306")

    conn = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db=db,
        port=port,
        charset='utf8',
        connect_timeout=60,
        read_timeout=300,
        write_timeout=300
    )

    print(f"\n>>> DB 연결 성공: {db}\n")
    return conn

def get_table_list(cursor, db_name):
    cursor.execute(f"SHOW TABLES FROM `{db_name}`")
    return [row[0] for row in cursor.fetchall()]

def is_table_empty(cursor, db_name, table):
    try:
        cursor.execute(f"SELECT COUNT(*) FROM `{db_name}`.`{table}`")
        count = cursor.fetchone()[0]
        return count == 0
    except Exception as e:
        print(f"[ERROR] {table} 컬럼 조회 실패: {e}")
        return False

def main():
    conn = get_connection()
    cursor = conn.cursor()
    db_name = conn.db.decode() if isinstance(conn.db, bytes) else conn.db

    tables = get_table_list(cursor, db_name)
    empty_tables_result = {}

    for table in tables:
        if is_table_empty(cursor, db_name, table):
            print(f"프린 테이블: {table} (데이터 없음)")
            empty_tables_result[table] = {
                "startDate": None,
                "endDate": None,
                "week": {},
                "month": {},
                "year": {}
            }

    cursor.close()
    conn.close()

    output = json.dumps(empty_tables_result, indent=2, ensure_ascii=False)

    print("\n결과:")
    print(output)

    filename = f"db_empty_tables_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(output)

    print(f"\n분석 완료! 결과 파일: {filename}")
    print(f"파일 경로: {os.path.abspath(filename)}")

if __name__ == "__main__":
    main()



>>> DB 연결 성공: sc301web

프린 테이블: MDCL_RSV (데이터 없음)
프린 테이블: SC_LOG (데이터 없음)
프린 테이블: call_test (데이터 없음)
프린 테이블: insr_info (데이터 없음)
프린 테이블: md_board_cmt (데이터 없음)
프린 테이블: md_content_bnf (데이터 없음)
프린 테이블: md_event (데이터 없음)
프린 테이블: md_event_reg (데이터 없음)
프린 테이블: md_ld_consult_copy (데이터 없음)
프린 테이블: md_medical_team (데이터 없음)
프린 테이블: md_member_group (데이터 없음)
프린 테이블: md_page_common (데이터 없음)
프린 테이블: md_pay_list (데이터 없음)
프린 테이블: md_res_each (데이터 없음)
프린 테이블: md_res_medi (데이터 없음)
프린 테이블: mdcl_rsv (데이터 없음)
프린 테이블: nb_comments (데이터 없음)
프린 테이블: nb_files (데이터 없음)
프린 테이블: nb_post_likes (데이터 없음)
프린 테이블: rcpt_dtal_cntn (데이터 없음)
프린 테이블: rcpt_info (데이터 없음)
프린 테이블: rollpop (데이터 없음)
프린 테이블: sms (데이터 없음)
프린 테이블: status_change_log (데이터 없음)
프린 테이블: surveys_question_responses (데이터 없음)
프린 테이블: surveys_responses (데이터 없음)
[ERROR] user_info 컬럼 조회 실패: (1242, 'Subquery returns more than 1 row')

결과:
{
  "MDCL_RSV": {
    "startDate": null,
    "endDate": null,
    "week": {},
    "month": {},
    "year": {}
  },
  "SC_LOG"

## 

In [3]:
import json
import pandas as pd
import os

def flatten_json_to_dataframe(json_data):
    all_months = set()
    all_years = set()
    table_rows = []

    for table_name, table_data in json_data.items():
        if table_name == "skipped":
            continue

        months = table_data.get("month", {})
        years = table_data.get("year", {})

        all_months.update(months.keys())
        all_years.update(years.keys())

    all_months = sorted(all_months)
    all_years = sorted(all_years)
    columns = ["table"] + all_months + all_years + ["총합 (MB)"]

    for table_name, table_data in json_data.items():
        if table_name == "skipped":
            continue

        row = {"table": table_name}
        total = 0.0

        # 월 단위
        for m in all_months:
            val = table_data.get("month", {}).get(m, "")
            row[m] = val or ""

        # 연 단위 + 총합 계산
        for y in all_years:
            val = table_data.get("year", {}).get(y, "")
            row[y] = val or ""
            if val:
                try:
                    total += float(val.replace(",", ""))
                except ValueError:
                    pass

        row["총합 (MB)"] = f"{total:,.3f}" if total else ""
        table_rows.append(row)

    return pd.DataFrame(table_rows, columns=columns)

def main():
    filename = "db_empty_tables_20250620_163638.json"  # 파일명
    if not os.path.exists(filename):
        print(f"파일이 존재하지 않습니다: {filename}")
        return

    with open(filename, "r", encoding="utf-8") as f:
        json_data = json.load(f)

    df = flatten_json_to_dataframe(json_data)

    output_filename = "db_empty_datetime_size_summary.xlsx"
    df.to_excel(output_filename, index=False)

    print(f"\n✔ 엑셀 파일 생성 완료: {output_filename}")
    print(f"경로: {os.path.abspath(output_filename)}")

if __name__ == "__main__":
    main()



✔ 엑셀 파일 생성 완료: db_empty_datetime_size_summary.xlsx
경로: /Users/sc301/Desktop/yhy/project/lagacy_db/db_empty_datetime_size_summary.xlsx
